<a href="https://colab.research.google.com/github/nareshyadav2912/fakenewsdetection/blob/main/Fake_News_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the Libraries

In [179]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

In [180]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [181]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [182]:
dataset = pd.read_csv('/content/drive/MyDrive/train.csv')
dataset.head(10)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,NaN,Ever wonder how Britain’s most iconic pop pian...,1
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
8,8,Excerpts From a Draft Script for Donald Trump’...,NaN,Donald J. Trump is scheduled to make a highly ...,0
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0


In [183]:
dataset.shape

(20800, 5)

In [184]:
dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [185]:
dataset1=dataset.fillna("")
dataset1.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [186]:
dataset1['content']=dataset1['author']+' '+dataset1['title']
dataset1['content'].head()

0    Darrell Lucus House Dem Aide: We Didn’t Even S...
1    Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2    Consortiumnews.com Why the Truth Might Get You...
3    Jessica Purkiss 15 Civilians Killed In Single ...
4    Howard Portnoy Iranian woman jailed for fictio...
Name: content, dtype: object

In [187]:
X=dataset1.drop('label',axis=1)
Y=dataset1['label']
#Y.head()
X.head()

,id,title,author,text,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,Howard Portnoy Iranian woman jailed for fictio...


In [188]:
X.shape,Y.shape

((20800, 5), (20800,))

In [189]:
ps=PorterStemmer()

In [190]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
  # sub substitues some values, numbers and space will be substituted by space in content
  # it is a regular expression syntax
  stemmed_content = stemmed_content.lower()
  # makes words to lowercase
  stemmed_content = stemmed_content.split()
  # splits the content 
  stemmed_content = [ps.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [ ]:
dataset1['content']=dataset1['content'].apply(stemming)
dataset1['content']

In [ ]:
X=dataset1['content'].values
Y=dataset1['label'].values

In [ ]:
print(X)

In [ ]:
print(Y)

In [ ]:
vector=TfidfVectorizer()
vector.fit(X)
X=vector.transform(X)

In [ ]:
print(X)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.3,stratify = Y, random_state =1000)

In [ ]:
lr=LogisticRegression()
lr.fit(X_train,Y_train)

In [ ]:
X_train_pred = lr.predict(X_train)
td_ac = accuracy_score(X_train_pred, Y_train )

In [ ]:
print(" Accuracy score for the traing data : ",td_ac*100)

In [ ]:
X_test_pred = lr.predict(X_test)
td_ac = accuracy_score(X_test_pred, Y_test )

In [ ]:
print(" Accuracy score for the testing data : ",td_ac*100)

In [ ]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import itertools
cm=confusion_matrix(Y_test,X_test_pred)
ConfusionMatrixDisplay(cm).plot()

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Y_test,X_test_pred))

In [ ]:
from sklearn.svm import LinearSVC

In [ ]:
sv=LinearSVC()
sv.fit(X_train,Y_train)

In [ ]:
X_train_pred = sv.predict(X_train)
td_ac = accuracy_score(X_train_pred, Y_train)

In [ ]:
print(" Accuracy score for the traing data : ",td_ac*100)

In [ ]:
X_test_pred = sv.predict(X_test)
td_ac = accuracy_score(X_test_pred, Y_test )

In [ ]:
print(" Accuracy score for the testing data : ",td_ac*100)

In [ ]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import itertools
cm=confusion_matrix(Y_test,X_test_pred)
ConfusionMatrixDisplay(cm).plot()

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Y_test,X_test_pred))

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
nv=MultinomialNB()
nv.fit(X_train,Y_train)

In [ ]:
X_train_pred = nv.predict(X_train)
td_ac = accuracy_score(X_train_pred, Y_train)

In [ ]:
print(" Accuracy score for the traing data : ",td_ac*100)

In [ ]:
X_test_pred = nv.predict(X_test)
td_ac = accuracy_score(X_test_pred, Y_test )

In [ ]:
print(" Accuracy score for the testing data : ",td_ac*100)

In [ ]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import itertools
cm=confusion_matrix(Y_test,X_test_pred)
ConfusionMatrixDisplay(cm).plot()

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Y_test,X_test_pred))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
data = {'Logisitc Regression':97, 'Support Vector Machine':98, 'Naive Bayes':95}
courses = list(data.keys())
values = list(data.values())
fig = plt.figure(figsize = (10, 5))
plt.bar(courses, values, color ='blue',width = 0.5)
plt.xlabel("Classification Algorithms")
plt.ylabel("Percentage of Accuracy")
plt.title("Accuracy of Classification Algorithms")
plt.show()
